In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

22/08/13 17:32:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/13 17:32:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/13 17:32:19 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/08/13 17:32:19 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/08/13 17:32:19 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [2]:

# 4
olist_order_payments_dataset = spark.read.options(header='True', inferSchema='True', delimiter=',') \
                            .csv("data/olist_order_payments_dataset.csv")
# 6
olist_orders_dataset = spark.read.options(header='True', inferSchema='True', delimiter=',') \
                            .csv("data/olist_orders_dataset.csv")

In [9]:
payments = olist_order_payments_dataset.select(col("order_id").alias("order_id_2"), "payment_value")
payments.show()

+--------------------+-------------+
|          order_id_2|payment_value|
+--------------------+-------------+
|b81ef226f3fe1789b...|        99.33|
|a9810da82917af2d9...|        24.39|
|25e8ea4e93396b6fa...|        65.71|
|ba78997921bbcdc13...|       107.78|
|42fdf880ba16b47b5...|       128.45|
|298fcdf1f73eb413e...|        96.12|
|771ee386b001f0620...|        81.16|
|3d7239c394a212faa...|        51.84|
|1f78449c87a54faf9...|       341.09|
|0573b5e23cbd79800...|        51.95|
|d88e0d5fa41661ce0...|       188.73|
|2480f727e869fdeb3...|        141.9|
|616105c9352a9668c...|        75.78|
|cf95215a722f3ebf2...|       102.66|
|769214176682788a9...|       105.28|
|12e5cfe0e4716b59a...|       157.45|
|61059985a6fc0ad64...|       132.04|
|79da3f5fe31ad1e45...|        98.94|
|8ac09207f415d55ac...|       244.15|
|b2349a3f20dfbeef6...|       136.71|
+--------------------+-------------+
only showing top 20 rows



In [10]:
olist_orders_dataset.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [13]:
join_table = payments.join(olist_orders_dataset, payments.order_id_2==olist_orders_dataset.order_id, "Inner")
join_table = join_table.select("order_id","payment_value","customer_id", "order_purchase_timestamp")
join_table = join_table.withColumn('order_purchase_timestamp',to_date(unix_timestamp \
            (join_table.order_purchase_timestamp, 'yyyy-MM-dd HH:mm:ss').cast('timestamp')))
join_table.show()

+--------------------+-------------+--------------------+------------------------+
|            order_id|payment_value|         customer_id|order_purchase_timestamp|
+--------------------+-------------+--------------------+------------------------+
|e481f51cbdc54678b...|        18.59|9ef432eb625129730...|              2017-10-02|
|e481f51cbdc54678b...|          2.0|9ef432eb625129730...|              2017-10-02|
|e481f51cbdc54678b...|        18.12|9ef432eb625129730...|              2017-10-02|
|53cdb2fc8bc7dce0b...|       141.46|b0830fb4747a6c6d2...|              2018-07-24|
|47770eb9100c2d0c4...|       179.12|41ce2a54c0b03bf34...|              2018-08-08|
|949d5b44dbf5de918...|         72.2|f88197465ea7920ad...|              2017-11-18|
|ad21c59c0840e6cb8...|        28.62|8ab97904e6daea886...|              2018-02-13|
|a4591c265e18cb1dc...|       175.26|503740e9ca751ccdd...|              2017-07-09|
|136cce7faa42fdb2c...|        65.95|ed0271e0b7da060a3...|              2017-04-11|
|651

In [15]:
join_table.createOrReplaceTempView("join_table")

#### 3)	Find maximum spending customers for every month of last year.

What mean last year when the newest data is from 2018?   
Last year of existing data?

In [22]:
spark.sql("""
            SELECT
            max(order_purchase_timestamp)
            FROM join_table
            """).show()

+-----------------------------+
|max(order_purchase_timestamp)|
+-----------------------------+
|                   2018-10-17|
+-----------------------------+



In [14]:
join_table.dtypes

[('order_id', 'string'),
 ('payment_value', 'double'),
 ('customer_id', 'string'),
 ('order_purchase_timestamp', 'date')]

In [19]:
df = spark.sql("""
               SELECT
               customer_id,
               SUM(payment_value)
               FROM join_table
               GROUP BY customer_id
               
               
               
               
               """)
df.show()

+--------------------+------------------+
|         customer_id|sum(payment_value)|
+--------------------+------------------+
|f54a9f0e6b351c431...|             35.95|
|2a1dfb647f32f4390...|            508.17|
|4f28355e5c17a4a42...|            153.72|
|4632eb5a8f175f6fe...|             91.66|
|843ff05b30ce4f75b...|            102.74|
|a4156bb8aff5d6722...|            206.87|
|1099d033c74a027a7...|              61.7|
|7a3bd3b37285f0ab2...|            473.12|
|5fff39f1b59dc4d2f...|             90.58|
|18f6ca10777417c93...|             61.36|
|f1e46939e6408b3e6...|              53.0|
|a8695124db570d100...|             85.24|
|545b9a267af9ba134...|            130.83|
|90d7075599361b694...|             84.28|
|a340ce6c3570e68d4...|            122.69|
|ce0681e1c3f70e145...|             82.73|
|ee905ec97794ec6e9...|170.23000000000002|
|b74ca180d63f9ae04...|             98.62|
|0de46efc7d10114ac...|             124.8|
|8bdae6b4ff9bc7f4c...|            376.56|
+--------------------+------------